In [ ]:
!pip install ISR

In [ ]:
!pip install 'h5py==2.10.0' --force-reinstall

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/image-super-resolution-master/

In [ ]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19
from ISR.train import Trainer
from ISR.utils import metrics
from PIL import Image
import numpy as np
import os

In [ ]:
import yaml
yaml.__version__

In [ ]:
!pip install -U PyYAML

# Training

In [ ]:
lr_train_patch_size = 50
layers_to_extract = [5, 9]
scale = 4
hr_train_patch_size = lr_train_patch_size * scale


rrdn = RRDN(arch_params={'C': 4, 'D':3, 'G':32, 'G0':32, 'T': 10, 'x':4}, patch_size=lr_train_patch_size)
# rrdn.model.load_weights('/content/drive/MyDrive/image-super-resolution-master/weights/rrdn-C4-D3-G32-G032-T10-x4/2021-08-02_1221/rrdn-C4-D3-G32-G032-T10-x4_best-val_generator_PSNR_Y_epoch026.hdf5')
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

In [ ]:
loss_weights = {
  'generator': 1,
  'feature_extractor': 0.006,
  'discriminator': 0.0
}
losses = {
  'generator': 'mse',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': './logs', 'weights': './weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30}

flatness = {'min': 0.0, 'max': 0.15, 'increase': 0.01, 'increase_frequency': 5}

trainer = Trainer(
    generator = rrdn,
    discriminator = discr,
    feature_extractor = f_ext,
    lr_train_dir = '/content/drive/MyDrive/Brain_data/LR_high_SNR/',
    hr_train_dir = '/content/drive/MyDrive/Brain_data/HR_high_SNR/',
    lr_valid_dir = '/content/drive/MyDrive/Brain_data/LR_high_SNR_val/',
    hr_valid_dir = '/content/drive/MyDrive/Brain_data/HR_high_SNR_val/',
    loss_weights = loss_weights,
    learning_rate = learning_rate,
    flatness = flatness,
    dataname = 'Brain_high_SNR',
    log_dirs = log_dirs,
    weights_generator = None,
    weights_discriminator = None,
    n_validation = 100,
)

In [ ]:
trainer.train(
    epochs = 30,
    steps_per_epoch = 120,
    batch_size = 8,
    monitored_metrics={'val_generator_PSNR_Y': 'max'}
)

# Quick visualisation

In [ ]:
rrdn = RRDN(arch_params={'C': 4, 'D':3, 'G':32, 'G0':32, 'T': 10, 'x':4})
rrdn.model.load_weights('/content/drive/MyDrive/image-super-resolution-master/weights/rrdn-C4-D3-G32-G032-T10-x4/2021-08-03_1225/rrdn-C4-D3-G32-G032-T10-x4_epoch100.hdf5')

In [ ]:
filelist = os.listdir('/content/drive/MyDrive/Brain_data/HR_high_SNR/')

In [ ]:
filelist[0]

In [ ]:
img = Image.open('/content/drive/MyDrive/Brain_data/HR_high_SNR/brain_peak0.png')
img

In [ ]:
img = Image.open('/content/drive/MyDrive/Brain_data/LR_high_SNR/brain_peak0.png')
img

In [ ]:
sr_img = rrdn.predict(np.array(img))
res = Image.fromarray(sr_img)
res